In [ ]:
# export
import traitlets
import json
import base64
import ipyvuetify
import ipywidgets
from nbdev.imports import *
import pandas as pd
from markdown import markdown
from vvapp.inputs import *

In [ ]:
# default_exp outputs

In [ ]:
# export
import ipyvuetify

# vvapp.outputs

>  Ipyvuetify Jupyter output widgets with a simplified API

In [ ]:
#hide
from nbdev.imports import *
from nbdev.showdoc import *

## Pandas DataFrame Output

In [ ]:


def df2download(htmlWidget, filename, df):
    """  Convert a pandas dataframe to a downloadaable link


    htmlWidget:  html ipywidget
    filename:  str,  Name of csv file
    df: pandas dataframe

    For example:
        htmlWidget = widgets.HTML(value = '')
        htmlWidget

        df2download(htmlWidget, 'selectedRows.csv', test.get_selected_df())

    """
    if isinstance(df, pd.DataFrame):
        payload = base64.b64encode(df.to_csv().encode()).decode()

        # Create and assign html to widget
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        #     html = '<button type="button">Click Me!</button><download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        htmlWidget.value = html.format(payload=payload,
                                       title=filename,
                                       filename=filename)
    else:
        htmlWidget.value = ''


class VuetifyTable_pandas_df(ipyvuetify.VuetifyTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame with similar methods to QGrid
    
    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
        multiple (Bool) - True (default) enables multiple line selection
        dense (Bool) - True (default) enables dense rendering of table



    Example:

        import pandas as pd

        iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
        test = VuetifyTable_pandas_df(df = iris)

        test.title = 'sdvsd'
        test.multiple = True
        test.dense = True

        def onChange_table(change,widget):
        #     print(change['new'])
            print(widget.value)

        test.observe(onChange_table,'value',test)

        test.get_selected_df()

        test.get_selected_rows()

    """

    headers = traitlets.List([]).tag(sync=True, allow_null=True)
    items = traitlets.List([]).tag(sync=True, allow_null=True)
    value = traitlets.List([]).tag(sync=True, allow_null=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('').tag(sync=True)
    index_col = traitlets.Unicode('').tag(sync=True)
    base64_data = traitlets.Unicode('').tag(sync=True)
    multiple = traitlets.Bool(False).tag(sync=True)
    show_select = traitlets.Bool(False).tag(sync=True)
    dense = traitlets.Bool(False).tag(sync=True)
    template = traitlets.Unicode('''
        <template>
          <v-card>
            <v-card-title>
              <span class="title font-weight-bold">{{ title }}</span>
              <v-spacer></v-spacer>
                    <v-text-field
                    v-model="search"
                    append-icon="mdi-magnify"
                    label="Search ..."
                    single-line
                    hide-details
                ></v-text-field>
                    
            </v-card-title>
            <v-data-table
                :headers="headers"
                :items="items"
                :search="search"
                :item-key="index_col"
                :single-select="multiple"
                :dense ="dense"
                :show-select="show_select"
                v-model="value"
                
                
    :footer-props="{
      showFirstLastPage: true,
      firstIcon: 'mdi-arrow-collapse-left',
      lastIcon: 'mdi-arrow-collapse-right',
      prevIcon: 'mdi-minus',
      nextIcon: 'mdi-plus',
      
    }"            >
                <template v-slot:no-data>
                  <v-alert :value="true" color="error" icon="mdi-alert">
                    Sorry, nothing to display here :(
                  </v-alert>
                </template>
                <template v-slot:no-results>
                    <v-alert :value="true" color="error" icon="mdi-alert">
                      Your search for "{{ search }}" found no results.
                    </v-alert>
                </template>
                <template v-slot:items="rows">
                  <td v-for="(element, label, index) in rows.item"
                      @click=cell_click(element)
                      >
                    {{ element }}
                  </td>
                </template>
            </v-data-table>
          </v-card>
        </template>
        ''').tag(sync=True)

    def __init__(self,
                 *args,
                 df=pd.DataFrame(),
                 title=None,
                 multiple=True,
                 dense=False,
                 **kwargs):
        super().__init__(*args, **kwargs)
        df = df.reset_index()
        self.index_col = df.columns[0]
        headers = [{"text": col, "value": col} for col in df.columns]
        headers[0].update({'align': 'left', 'sortable': True})
        self.headers = headers
        self.items = json.loads(df.to_json(orient='records'))
        if title is not None:
            self.title = title

        self.dense = dense
        self.multiple = multiple

        self.htmlWidget = ipywidgets.HTML(value='')
        self.observe(self.onChange_table)
        self.base64_data=""

        # Combine table and widgets
        self.widgets = ipyvuetify.Card(children=[
            ipywidgets.VBox([
                self,
                ipyvuetify.Row(children=[
                    ipyvuetify.Tooltip(
                        bottom=True,
                        v_slots=[{
                            'name':'activator',
                            'variable':'tooltip',
                            'children':
                            ipyvuetify.Btn(v_on='tooltip.on',
                                           color='primary',
                                           class_='ma-3',
                                           icon=True,
                                           children=[
                                               ipyvuetify.Icon(
                                                   centre=True,
                                                   medium=True,
                                                   children=['mdi-information'])
                                           ])
                        }],
                        children=[
                            'Click the link to the right to download table as CSV'
                        ]), self.htmlWidget
                ],
                               align="center",
                               justify="left")
            ])
        ])

    def get_selected_df(self):
        df = pd.DataFrame.from_dict(self.items)
        df = df.set_index('index')
        df.base64_data = base64.b64encode(df.to_csv().encode()).decode()
        return df

    def get_selected_rows(self):
        return [dictionary['index'] for dictionary in self.value]

    def onChange_table(self, change):
        df2download(self.htmlWidget, 'download_data.csv',
                    self.get_selected_df())


#penguins = pd.read_csv(
#    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv')
penguins = pd.DataFrame({'a':[1,2,3],'b':[2,3,4]})
test = VuetifyTable_pandas_df(df=penguins)
test.title = 'Penguins Data'
test.multiple = False
test.dense = True


# example function when table changes
#def onChange_table(change, widget):
#    print(widget.value)

#test.observe(
#    onChange_table,
##    'value',
#   test,
#)

test.widgets

Card(children=[VBox(children=(VuetifyTable_pandas_df(dense=True, headers=[{'text': 'index', 'value': 'index', …

In [ ]:
#export

class output_pandas_table(ipyvuetify.VuetifyTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame

    Args:
        data (pandas.DataFrame) - the data to render
        title (str) - optional title

    Modified from Source: https://jupyter-tutorial.readthedocs.io/de/latest/workspace/jupyter/ipywidgets/libs/ipyvuetify.html
    """

    headers = traitlets.List([]).tag(sync=True, allow_null=True)
    items = traitlets.List([]).tag(sync=True, allow_null=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    index_col = traitlets.Unicode('').tag(sync=True)
    template = traitlets.Unicode('''
        <template>
          <v-card>
            <v-card-title>
              <span class="title font-weight-bold">{{ title }}</span>
              <v-spacer></v-spacer>
                <v-text-field
                    v-model="search"
                    append-icon="search"
                    label="Search ..."
                    single-line
                    hide-details
                ></v-text-field>
            </v-card-title>
            <v-data-table
                :headers="headers"
                :items="items"
                :search="search"
                :item-key="index_col"
                :rows-per-page-items="[25, 50, 250, 500]"
                multiple
            >
                <template v-slot:no-data>
                  <v-alert :value="true" color="error" icon="warning">
                    Sorry, nothing to display here :(
                  </v-alert>
                </template>
                <template v-slot:no-results>
                    <v-alert :value="true" color="warning" icon="warning">
                      Your search for "{{ search }}" found no results.
                    </v-alert>
                </template>
                <template v-slot:items="rows">
                  <td v-for="(element, label, index) in rows.item"
                      @click=cell_click(element)
                      >
                    {{ element }}
                  </td>
                </template>
            </v-data-table>
          </v-card>
        </template>
        ''').tag(sync=True)

    def __init__(self, *args,
                 data=pd.DataFrame(),
                 title=None,
                 **kwargs):
        super().__init__(*args, **kwargs)
        data = data.reset_index()
        self.index_col = data.columns[0]
        headers = [{
              "text": col,
              "value": col
            } for col in data.columns]
        headers[0].update({'align': 'left', 'sortable': True})
        self.headers = headers
        self.items = json.loads(
            data.to_json(orient='records'))
        if title is not None:
            self.title = title

In [ ]:
df = pd.DataFrame({'a':[1,2,3],'b':[2,3,4]})
output_pandas_table(data=df,title='My DataFrame')

output_pandas_table(headers=[{'text': 'index', 'value': 'index', 'align': 'left', 'sortable': True}, {'text': …

## Markdown Output

In [ ]:
#export 

def output_markdown(*args, **kwargs):
    """
    Render input string(s) as markdown

    Parameters
    *args : str or list of strings
        String(s) to render as markdown
    **kwargs : any
        arguments supported by the markdown class

    """
    return ipywidgets.HTML(
        markdown(
            '\n'.join(args),
            **kwargs
        )
    )

In [ ]:
output_markdown("""
# Markdown Title

## Markdown Subtitle

Some body text

* a list element
* another list element
""")

HTML(value='<h1>Markdown Title</h1>\n<h2>Markdown Subtitle</h2>\n<p>Some body text</p>\n<ul>\n<li>a list eleme…

## Snackbar

In [ ]:
# export

class _InfoSnackbar(ipyvuetify.VuetifyTemplate):
    color = traitlets.Unicode("primary").tag(sync=True, allow_null=True)
    message = traitlets.Unicode("").tag(sync=True, allow_null=False)
    timeout = traitlets.Integer(5000).tag(sync=True, allow_null=True)
    active = traitlets.Bool(True).tag(sync=True, allow_null=True)
    multi_line = traitlets.Bool(True).tag(sync=True, allow_null=True)

    template = traitlets.Unicode(
        """
        <template>
            <v-snackbar
                v-model="active"
                centered=true
                :color="color"
                elevation=0
                :multi-line="multi_line"
                :timeout="timeout"
            >
              {{message}}

            <v-btn
              color="white"
              text
              @click="active = false"
            >
              Close
            </v-btn>

            </v-snackbar>
          </template>

        """
    ).tag(sync=True)

    def __init__(
        self,
        snackbar=False,
        message=None,
        multi_line=True,
        color="primary",
        timeout=6000,
        active=True,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.color = color
        self.timeout = timeout
        self.active = active
        self.message = message
        self.multi_line = multi_line

def output_info_snackbar(snackbar=False,
        message=None,
        multi_line=True,
        color="primary",
        timeout=6000,
        active=True):
    """
    snackbar widget

    Useful for displaying information/messages

    message : str
        Status message to display
    timeout : int, optional default 6000
        How long to display the message (in milliseconds)
    multi_line : bool, optional default True
        Whether the snackbar should be multi-line (or not)
    color : str, optional default 'primary'
        ipyvuetify color of snackbar
        Typical options include ['primary','error','warning','success']
    active : bool, optional, default True
        Whether to display snackbar
        """
    return _InfoSnackbar(snackbar=snackbar,
                        message=message,
                        multi_line=multi_line,
                        color=color,
                        timeout=timeout,
                        active=active)

In [ ]:
output_info_snackbar(message='Message text', color='warning',timeout=4000, multi_line=True)

_InfoSnackbar(color='warning', message='Message text', timeout=4000)

## Dialog Button

In [ ]:
# export


def output_dialog_button(label='',
                  icon='mdi-help',
                  _class="icon ma-2",
                  _style="",
                  color="primary",
                  dark=False,
                  dialog_width=600,
                  dialog_title="Dialog Title",
                  dialog_content="Dialog Content",
                  **args
                ):
    """
    Creates a button and activates a dialog on click

    Useful to display application documentation/help

    Parameters
    ----------
    icon : str (optional, default None)
        Icon to display on button
    label : str (optoinal, default None)
        Text to display on button
    dialog_width : int (optional, default 600)
        Width of the dialog in pixels
    dialog_title : str (optional, default 'Help Title')
        Dialog title
    dialog_content : str (optional, default '')
        Dialog content in markdown format
    color : str (optional, default 'primary')
        Color of button
    dark : bool
        Use dark style
    _class : str (optional, default 'icon ma-2')
        CSS classes of button
    _style: str
        CSS style of button
    **args
        Other arguments will be passed on to ipyvuetify.Dialog
    """

    if icon is None:
        icon=''

    if label is None:
        label=''

    dialog_button = ipyvuetify.Btn(
        _class_="icon ma-2",
        _style='',
        color=color,
        depressed=True,
        dark=dark,
        children=[label, ipyvuetify.Icon(children=[icon])],
    )

    close_dialog_btn = ipyvuetify.Btn(children=["Close"])

    dialog_dialog = ipyvuetify.Dialog(
        v_model=False,
        scrollable=True,
        width=dialog_width,
        children=[
            ipyvuetify.Card(children=[
                ipyvuetify.CardTitle(class_="headline pa-4",
                                     children=[dialog_title]),
                ipyvuetify.Divider(),
                ipyvuetify.CardText(
                    class_="pa-4 text--primary",
                    primary_title=True,
                    children=[dialog_content],
                ),
                ipyvuetify.Divider(),
                close_dialog_btn,
            ])
        ],
        **args
    )

    def open_dialog_dialog(widget, event, data):
        dialog_dialog.v_model = True

    def close_dialog_dialog(widget, event, data):
        dialog_dialog.v_model = False

    display(ipyvuetify.Layout(children=[dialog_dialog]))
    close_dialog_btn.on_event("click", close_dialog_dialog)
    dialog_button.on_event("click", open_dialog_dialog)
    return dialog_button

In [ ]:
output_dialog_button(label='My Button',icon=None)

Layout(children=[Dialog(children=[Card(children=[CardTitle(children=['Dialog Title'], class_='headline pa-4'),…

Btn(children=['My Button', Icon(children=[''])], color='primary', dark=False, depressed=True)

## Container

In [ ]:
# export


def output_container(
    children=[],
    fluid=False,
    _class="icon ma-2",
    _style="",
    **args
):
    """
    Creates a button and activates a dialog on click

    Useful to display application documentation/help

    Parameters
    ----------
    children : list
        List of elements to display in container
    fluid : bool (default False)
        Removes viewport maximum-width size breakpoints
    _class : str (optional, default 'icon ma-2')
        CSS classes of button
    _style: str
        CSS style of button
    **args
        Other arguments will be passed on to ipyvuetify.Container
    """

    ret = ipyvuetify.Container(
        _class_=_class,
        _style=_style,
        children=children,
        fluid=fluid,
        **args
    )

    return ret

In [ ]:
output_container(fluid=False,
          children=[
              input_button(color='red', label='Red'),
              input_button(color='blue', label='Blue'),
              input_button(color='green', label='Green')
          ])

Container(children=[Btn(children=['Red'], color='red', dark=False, fab=False, large=False, outlined=False, rou…

## Row

In [ ]:
# export


def output_row(
    children=[],
    align=None,
    align_content=None,
    justify=None,
    dense=False,
    no_gutters=False,
    _class="",
    _style="",
    **args
):
    """
    Create a row output container

    For details see: https://vuetifyjs.com/en/components/grids/

    Parameters
    ----------
    children : list (default [])
        List of elements to display in container
    align : str (default None)
        Applies the align-items css property. Available options are start, center, end, baseline and stretch.
    align_content : str (default None)
        Applies the align-content css property. Available options are start, center, end, space-between, space-around and stretch.
    justify : str (default None)
        Applies the justify-content css property. Available options are start, center, end, space-between and space-around.
    dense : bool (default False)
        Reduces the gutter between v-cols.
    no_gutters : bool (default False)
        Removes the gutter between v-cols.
    _class : str (optional, default 'icon ma-2')
        CSS classes of button
    _style: str
        CSS style of button
    **args
        Other arguments will be passed on to ipyvuetify.Row
    """

    ret = ipyvuetify.Row(
        children=children,
        align=align,
        align_content=align_content,
        justify=justify,
        dense=dense,
        no_gutters=no_gutters,
        _class_=_class,
        _style=_style,
        **args
    )

    return ret

In [ ]:
output_container(children=[
    output_row(children=[input_button(color='red', label='Red')]),
    output_row(children=[input_button(color='blue', label='Blue')]),
    output_row(children=[input_button(color='green', label='Green')])
])

Container(children=[Row(children=[Btn(children=['Red'], color='red', dark=False, fab=False, large=False, outli…

## Column

In [ ]:
# export


def output_column(
    children=[],
    cols=None,
    offset=None,
    order=None,
    align_self=None,
    class_="",
    style_="",
    **args
):
    """
    Create a column output container

    For details see: https://vuetifyjs.com/en/components/grids/

    Parameters
    ----------
    children : list (default [])
        List of elements to display in container
    cols : None, str, int
        Sets the default number of columns the component extends. Available options are 1 -> 12 and auto.
    offset : None, str, int
        Sets the default offset for the column.
    order: None
        Sets the default order for the column.
    align_self : None, str, int
        Applies the align-items css property. Available options are start, center, end, auto, baseline and stretch. See https://developer.mozilla.org/en-US/docs/Web/CSS/align-items
    class_ : str (optional, default 'icon ma-2')
        CSS classes of button
    style_: str
        CSS style of button
    **args
        Other arguments will be passed on to ipyvuetify.Column
    """

    ret = ipyvuetify.Col(
        children=children,
        cols=cols,
        offset=offset,
        order=order,
        align_self=align_self,
        class_=class_,
        style_=style_,
        justify_space_between=True,
    )

    return ret

In [ ]:
output_row(children=[
    output_column(order=1, cols=4, children=[input_button(color='red', label='Red')]),
    output_column(order=0, cols=4, children=[input_button(color='blue', label='Blue')]),
    output_column(order=2, cols=4, children=[input_button(color='green', label='Green')])
])

Row(children=[Col(children=[Btn(children=['Red'], color='red', dark=False, fab=False, large=False, outlined=Fa…

In [ ]:
#hide
from nbdev.export import notebook2script 
notebook2script()

Converted 00_inputs.ipynb.
Converted 01_outputs.ipynb.
Converted 02_templates.ipynb.
Converted index.ipynb.
